The first step in using Spark is to connecting to a `cluster`. In practice, the cluster is hosted on a remote machine that's connected to all other nodes. To create a connection we need to create an instance of `SparkContext` class. The class constructor takes in few optional arguments that allows to specify the attributes of the cluster. An object holding these attributes can be created using `SparkConf` class.

### Examining the Spark Context

here `SparkContext` object `sc` is already loaded into the workspace

In [1]:
from pyspark import SparkConf, SparkContext

configure = SparkConf().setAppName("example-app").setMaster("local")
sc = SparkContext(conf=configure)

In [2]:
print(sc)
print(sc.version)

<SparkContext master=local appName=example-app>
2.4.0


In [3]:
flights = sc.parallelize('flights_small.csv')

Spark's core data structure is `Resilient Distributed Dataset` (RDD). This is a low level object that let's Spark work its magic by splitting data across multiple nodes in the cluster. Spark DataFrame behaves lot like SQL table. To start working on Spark DataFrames we have to create `SparkSession` object from `SparkContext` object

`SparkContext` is like to connection to the cluster and `SparkSession` is like an interface with that connection

### Creating a SparkSession

In [4]:
from pyspark.sql import SparkSession

my_spark = SparkSession.builder.getOrCreate()
print(my_spark)

In [5]:
my_spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.port', '56879'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'DESKTOP-K2G4QJD'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'example-app'),
 ('spark.app.id', 'local-1554318054003')]

### Loading Data

In [6]:
flights = my_spark.read.csv("flights_small.csv", header=True)

In [7]:
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

### Converting Spark DataFrame to Pandas DataFrame

In [8]:
flights_pd = flights.toPandas()

In [9]:
flights_pd.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43
3,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83,569,17,5
4,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127,937,7,54


### Converting Pandas DataFrame to Spark DataFrame

In [12]:
spark_flights = my_spark.createDataFrame(flights_pd)

now we will register the Spark DataFrame into a table in the Spark catalog

In [14]:
spark_flights.createOrReplaceTempView("spark_flights")

### Viewing tables

In [20]:
my_spark.catalog.listTables()

[Table(name='spark_flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In the above tables, we can run `SQL query` to fetch the data. For example - 

In [16]:
query = "FROM spark_flights SELECT * LIMIT 10"
my_flights = my_spark.sql(query)

### Creating columns

In [ ]:
spark_flights.count()

In [22]:
flights = my_spark.table('spark_flights')

In [23]:
flights = flights.withColumn("duration_hrs", flights.air_time / 60)

In [24]:
flights.show()

Py4JJavaError: An error occurred while calling o166.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.